In [1]:
import pandas as pd
import os
import glob
from typing_extensions import Concatenate
import cv2
import albumentations as A
import numpy as np
from PIL import Image
from albumentations.augmentations.geometric import OpticalDistortion

In [2]:
cv2.__version__
A.__version__

'1.3.0'

## 60살 augmentation

In [3]:
# CenterCrop = A.Compose([
#     A.CenterCrop(width=288, height=384, p=1)
# ])

# ColorJitter = A.Compose([
#     A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=True, p=1)
# ])
# OpticalDistortion =A.Compose([
#     OpticalDistortion(distort_limit=2.5, shift_limit=0.9, interpolation=1, border_mode=4, value=None, mask_value=None, p=1.0)
# ])

t1 = A.Compose([
    A.HorizontalFlip(p=1)    
])
t2 = A.Compose([
    A.Rotate(limit=[15,45], p=1, always_apply=True)
])
t3 = A.Compose([
    A.Rotate(limit=[-45,-15], p=1, always_apply=True)
])
t4 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=[15,45], p=1, always_apply=True)
])
t5 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=[-45,-15], p=1, always_apply=True)
])


In [4]:
root = '/opt/ml/input/data'

filenames = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

paths = []
df = pd.read_csv(os.path.join(root, 'train', 'train.csv'))
for path in df['path']:
    p = [glob.glob(os.path.join(root, 'train', 'images', path, f+'*')) for f in filenames]
    paths.extend(p)

In [ ]:
import re
def _parse(p):
        """
        path를 파싱해 라벨 리턴
        """
        p = str(p)
        match = re.search('_(.+)_Asian_(\d+)/(.*)[\.-]', p)
        if match and len(match.groups()) == 3:
            gender, age, mask = match.groups()
            gender = 0 if gender == 'male' else 1
            age = int(age)

            if mask.startswith('normal'):
                mask = 2
            elif mask.startswith('incorrect'):
                mask = 1
            else:
                mask = 0
            return mask, gender, age
        else:
            raise Exception(f'Cannot parsing label from the path: {p}')

In [5]:
print ("p : ", paths[0])
for p in paths:
    mask, gender, age = _parse(p[0])
    if age == 60:
        image = np.array(Image.open(p[0]))
        
        _, ext = p[0].split('.')
        new_path = p[0].replace('images', '60aug-images')
        
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        
        # Image.fromarray(t1(image=image)["image"]).save(new_path[:-4] + '_T1.' + ext)
        Image.fromarray(t2(image=image)["image"]).save(new_path[:-4] + '_T2.' + ext)
        Image.fromarray(t3(image=image)["image"]).save(new_path[:-4] + '_T3.' + ext)
        # Image.fromarray(t4(image=image)["image"]).save(new_path[:-4] + '_T4.' + ext)
        # Image.fromarray(t5(image=image)["image"]).save(new_path[:-4] + '_T5.' + ext)

p :  ['/opt/ml/input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg']


KeyboardInterrupt: 

In [5]:
root = '/opt/ml/input/data'

filenames = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

paths = []
df = pd.read_csv(os.path.join(root, 'train', 'train.csv'))
for path in df['path']:
    _, gender, _, age = path.split('_')
    if int(age) >= 60:
        full_paths = [glob.glob(os.path.join(root, 'train', 'images', path, filename+'*')) for filename in filenames]
        paths.extend(full_paths)
        # print(p)
        # p == [['/opt/ml/input/data/train/images/001038_male_Asian_60/incorrect_mask.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/mask1.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/mask2.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/mask3.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/mask4.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/mask5.jpg'], ['/opt/ml/input/data/train/images/001038_male_Asian_60/normal.jpg']]
        for full_path in full_paths:
            full_path = full_path[0]
            # full_path == '/opt/ml/input/data/train/images/001038_male_Asian_60/incorrect_mask.jpg'
            image = np.array(Image.open(full_path))
        
            _, ext = full_path.split('.')
            new_path = full_path.replace('images', '60aug_images')
            
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            Image.fromarray(t1(image=image)["image"]).save(new_path[:-4] + '_T1.' + ext) # Rotate(45)
            Image.fromarray(t2(image=image)["image"]).save(new_path[:-4] + '_T2.' + ext) # Rotate(45)
            Image.fromarray(t3(image=image)["image"]).save(new_path[:-4] + '_T3.' + ext) # Rotate(45)
            Image.fromarray(t4(image=image)["image"]).save(new_path[:-4] + '_T4.' + ext) # Rotate(45)
            Image.fromarray(t5(image=image)["image"]).save(new_path[:-4] + '_T5.' + ext) # Rotate(45)

## 60살 미만 + 마스크(Normal or Incorrect) 데이터 augmentation

In [4]:
t2 = A.Compose([
    A.Rotate(limit=[15,45], p=1, always_apply=True)
])
t3 = A.Compose([
    A.Rotate(limit=[-45,-15], p=1, always_apply=True)
])

In [6]:
root = '/opt/ml/input/data'

filenames = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

paths = []
df = pd.read_csv(os.path.join(root, 'train', 'train.csv'))
for path in df['path']:
    p = [glob.glob(os.path.join(root, 'train', 'images', path, f+'*')) for f in filenames]
    paths.extend(p)

In [ ]:
print(p[0])

/opt/ml/input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg


In [12]:
import re
def _parse(p):
        """
        path를 파싱해 라벨 리턴
        """
        p = str(p)
        match = re.search('_(.+)_Asian_(\d+)/(.*)[\.-]', p)
        if match and len(match.groups()) == 3:
            gender, age, mask = match.groups()
            gender = 0 if gender == 'male' else 1
            age = int(age)

            if mask.startswith('normal'):
                mask = 2
            elif mask.startswith('incorrect'):
                mask = 1
            else:
                mask = 0
            return mask, gender, age
        else:
            raise Exception(f'Cannot parsing label from the path: {p}')

In [13]:
print ("p : ", paths[0])
for p in paths:
    mask, gender, age = _parse(p[0])
    # print(age)
    if age < 60 and mask > 0: # mask : normal or incorrect
        image = np.array(Image.open(p[0]))
        
        _, ext = p[0].split('.')
        new_path = p[0].replace('images', 'notwear-incorrect-aug-images')
        
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        
        Image.fromarray(t2(image=image)["image"]).save(new_path[:-4] + '_T2.' + ext)
        Image.fromarray(t3(image=image)["image"]).save(new_path[:-4] + '_T3.' + ext)

p :  ['/opt/ml/input/data/train/images/000001_female_Asian_45/incorrect_mask.jpg']
45
45
45
45
45
45
45
52
52
52
52
52
52
52
54
54
54
54
54
54
54
58
58
58
58
58
58
58
59
59
59
59
59
59
59
58
58
58
58
58
58
58
58
58
58
58
58
58
58
56
56
56
56
56
56
56
58
58
58
58
58
58
58
57
57
57
57
57
57
57
58
58
58
58
58
58
58
55
55
55
55
55
55
55
58
58
58
58
58
58
58
59
59
59
59
59
59
59
57
57
57
57
57
57
57
50
50
50
50
50
50
50
54
54
54
54
54
54
54
58
58
58
58
58
58
58
55
55
55
55
55
55
55
58
58
58
58
58
58
58
49
49
49
49
49
49
49
56
56
56
56
56
56
56
58
58
58
58
58
58
58
58
58
58
58
58
58
58
55
55
55
55
55
55
55
54
54
54
54
54
54
54
57
57
57
57
57
57
57
55
55
55
55
55
55
55
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
57
57
57
57
57
57
57
57
57
57
57
57
57
58
58
58
58
58
58
58
56
56
56
56
56
56
56
58
58
58
58
58
58
58
56
56
56
56
56
56
56
55
55
55
55
55
55
55
58
58
58
58
58
58
58
57
57
57
57
57
57
57
55
55
55
55
55
55
55
59
59
59
59
59
59
59
53
53
53
53
53
53
53
54
54
54
54
54